In [1]:
#coding:utf-8
import pandas as pd
import time

from sklearn.metrics import log_loss
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
from scipy.stats import norm, skew #加载一些统计的库

In [2]:
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_: 
    :return: 
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))

In [3]:
def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('item_category_list_ing')
    for i in range(3):
        data['category_%d'%(i)] = data['item_category_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_category_list']

    print('item_property_list_ing')
    for i in range(3):
        data['property_%d'%(i)] = data['item_property_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_property_list']

    return data

In [4]:
data = pd.read_table('train.txt',sep=' ',index_col=False)
data2 = pd.read_table('test.txt',sep=' ',index_col=False) 
data3 = pd.read_table('test1.txt',sep=' ',index_col=False) 

In [5]:
data = pre_process(data)
data2 = pre_process(data2)
data3 = pre_process(data3)

item_category_list_ing
item_property_list_ing
item_category_list_ing
item_property_list_ing
item_category_list_ing
item_property_list_ing


## 统计职业对应类别

In [6]:
data[['user_occupation_id','is_trade']].groupby(['user_occupation_id']).count()

,is_trade
user_occupation_id,
-1,23890
2002,3539198
2003,144273
2004,583163
2005,6141512


In [7]:
data[['user_occupation_id','category_1','is_trade']].groupby(['user_occupation_id','category_1']).count()/data[['user_occupation_id','is_trade']].groupby(['user_occupation_id']).count()

is_trade
user_occupation_id category_1                   
-1                 1147074168968532252  0.006362
                   1367177154073382718  0.002763
                   1852600517265062354  0.050314
                   1909641874861640857  0.107451
                   1916390345133212703  0.021013
                   1920084168104334820  0.001339
                   2211060154630359130  0.034157
                   2871729383671301763  0.055630
                   3089254302947620489  0.002972
                   3348197449185791127  0.008623
                   3434689896486063330  0.028003
                   3613783563199627217  0.141231
                   394915394741096735   0.014525
                   453525480366550911   0.030306
                   4911723539855588624  0.024864
                   5066527928272238333  0.010674
                   5685690139879409547  0.058602
                   597424223456586363   0.000586
                   6254910033820815248  0.000167
                   6670526099037031245  0.116283
                   6693726201323251689  0.032357
                   7226013370341271704  0.004897
                   7314150500379498593  0.029552
                   7423553047267511438  0.001005
                   768579787521575291   0.118962
                   8009556227083201357  0.009544
                   8468007938333142982  0.001381
                   8468370105908620354  0.018711
                   8769426218101861255  0.002093
                   8841625760168847319  0.064378
...                                          ...
 2005              1367177154073382718  0.002873
                   1852600517265062354  0.049002
                   1909641874861640857  0.086861
                   1916390345133212703  0.024991
                   1920084168104334820  0.001198
                   2211060154630359130  0.035808
                   2871729383671301763  0.070607
                   3089254302947620489  0.004416
                   3348197449185791127  0.014617
                   3434689896486063330  0.028784
                   3613783563199627217  0.119723
                   394915394741096735   0.006214
                   453525480366550911   0.046823
                   4911723539855588624  0.036836
                   5066527928272238333  0.010136
                   5685690139879409547  0.064371
                   597424223456586363   0.001516
                   6254910033820815248  0.000114
                   6670526099037031245  0.071733
                   6693726201323251689  0.039422
                   7226013370341271704  0.006562
                   7314150500379498593  0.038428
                   7423553047267511438  0.002425
                   768579787521575291   0.134757
                   8009556227083201357  0.009820
                   8468007938333142982  0.002252
                   8468370105908620354  0.009916
                   8769426218101861255  0.003445
                   8841625760168847319  0.069776
                   926205401302902289   0.000409

[155 rows x 1 columns]

## 开始统计

In [8]:
all_data = data.append(data2)
all_data = all_data.append(data3)

In [9]:
occupation_click = all_data[['user_occupation_id','category_1','instance_id']].groupby(['user_occupation_id','category_1']).count()/data[['user_occupation_id','instance_id']].groupby(['user_occupation_id']).count()

In [10]:
occupation_click = occupation_click.rename(columns=({'instance_id':'occupation_click'}))

In [11]:
occupation_click = occupation_click.reset_index()

In [12]:
occupation_click

,user_occupation_id,category_1,occupation_click
0,-1,1147074168968532252,0.007158
1,-1,1367177154073382718,0.003181
2,-1,1852600517265062354,0.061992
3,-1,1909641874861640857,0.133403
4,-1,1916390345133212703,0.026455
5,-1,1920084168104334820,0.001632
6,-1,2211060154630359130,0.043282
7,-1,2871729383671301763,0.069108
8,-1,3089254302947620489,0.003809
9,-1,3348197449185791127,0.010088


In [13]:
occupationclickdata = pd.merge(data,occupation_click,on=['user_occupation_id','category_1'],how='left')

In [14]:
occupationclickdata2 = pd.merge(data2,occupation_click,on=['user_occupation_id','category_1'],how='left')

In [15]:
occupationclickdata3 = pd.merge(data3,occupation_click,on=['user_occupation_id','category_1'],how='left')

In [16]:
occupationclickdata2

,instance_id,item_id,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,user_gender_id,...,shop_score_service,shop_score_delivery,shop_score_description,category_0,category_1,category_2,property_0,property_1,property_2,occupation_click
0,93294255633855,2615021910672727866,3434999509929816124,196257267849351217,7,12,11,19,8447938236547496844,0,...,0.969971,0.969399,0.964373,836752724084922533,768579787521575291,,2636395404473730413,772325102984507165,1243768796732904674,0.150360
1,558322259509454,3748469724871724205,2274097622688424543,3122721854741763495,10,11,12,21,1644660685156574751,0,...,0.983832,0.987976,0.989656,836752724084922533,5685690139879409547,7497531498747093265,6241534230954727302,367082587220462692,2072967855524022579,0.088195
2,594573634113186,2111357467597282780,7838285046767229711,1019055478500227370,4,14,13,17,1899839434284249148,0,...,0.967112,0.967359,0.961470,836752724084922533,768579787521575291,,3540471119339706981,772325102984507165,251873273932348656,0.158018
3,667327653735176,3717651022681022321,1962186990434139903,2174699138227015967,7,11,11,16,4108506449488241435,0,...,0.946592,0.944660,0.930338,836752724084922533,2211060154630359130,7848078693615386447,2072967855524022579,2636395404473730413,4621934203383159480,0.041741
4,697732672924394,8284923347262844848,5390915051549817288,196257267849351217,6,12,12,18,1986129739312382401,0,...,0.977894,0.979554,0.981132,836752724084922533,1909641874861640857,,6241534230954727302,5977512434884267894,2072967855524022579,0.103088
5,1467522731866334,122990815260714082,8334914790811141908,6219110439660858399,6,14,13,18,6545279318527448248,0,...,0.956073,0.955764,0.980872,836752724084922533,8841625760168847319,,6241534230954727302,367082587220462692,5977512434884267894,0.081302
6,2564345731136577,5947872684354624624,7147251880558104750,6027631083130214337,8,12,11,19,6487822643782854186,0,...,0.977898,0.976857,0.984545,836752724084922533,2211060154630359130,7848078693615386447,6241534230954727302,367082587220462692,2072967855524022579,0.041741
7,2635932702100660,4376223620063302249,969027688928478372,8835551476805511776,9,9,11,16,1162841664798642252,0,...,0.979576,0.977407,0.988806,836752724084922533,3613783563199627217,6370392357088946840,6241534230954727302,367082587220462692,2072967855524022579,0.140320
8,3208825666465730,5226910727221958616,3601958674746901758,7534238860363577544,6,13,12,17,4693309019429064966,0,...,0.977467,0.972449,0.989476,836752724084922533,2871729383671301763,3492642177859571113,367082587220462692,5131280576272319091,8689439327673289769,0.082512
9,3654424870386483,1981860414878829638,1637947898625411891,6219110439660858399,6,12,11,19,3155709470897336062,0,...,0.962937,0.963305,0.955644,836752724084922533,1916390345133212703,,6241534230954727302,367082587220462692,5977512434884267894,0.027409


In [17]:
occupationclickdata[['instance_id','occupation_click']].to_csv('train_occupation_click.csv',index=False) 

In [18]:
occupationclickdata2[['instance_id','occupation_click']].to_csv('test_occupation_click.csv',index=False) 

In [19]:
occupationclickdata3[['instance_id','occupation_click']].to_csv('test1_occupation_click.csv',index=False) 

In [20]:
occupationclickdata3.shape

(1209768, 31)